<a href="https://colab.research.google.com/github/Dino-Boooo/Bio_meta_tran/blob/main/Bio_DeepEC_Molly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
GIT_REPO = 'https://github.com/kaistsystemsbiology/DeepProZyme.git'

# !rm -rf deepzyme
!git clone -b main --single-branch {GIT_REPO}

Cloning into 'DeepProZyme'...
remote: Enumerating objects: 1097, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1097 (delta 11), reused 10 (delta 10), pack-reused 1083 (from 2)
Receiving objects: 100% (1097/1097), 80.11 MiB | 24.87 MiB/s, done.
Resolving deltas: 100% (669/669), done.


In [3]:
# capture

##!pip install transformers==3.5.1 # install failed: version too old
##!pip install torch==1.7.0 # install failed: version too old
!pip install transformers
!pip install torch
!pip install logomaker

#imprort conda
!pip install -q condacolab
import condacolab
condacolab.install()

# !conda install -c conda-forge biopython=1.78
# !pip install transformers

##!conda install -c bioconda diamond=2.0.15 # version conflict: Use mamba instead
!mamba install -c bioconda diamond=2.0.15

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!diamond --version

diamond version 2.0.15


In [1]:
!pip install numpy==1.23.5 --force-reinstall
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 80.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-

In [ ]:
#@title Import modules
%cd /content/DeepProZyme

import os
import shutil
import logging

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

import logomaker

import sys
sys.path.append("/content/DeepProZyme/")

from deepec.data_loader import DeepECDataset
#from deepec.utils import argument_parser, run_neural_net, save_dl_result
#from deepec.homology import run_blastp, read_best_blast_result, merge_predictions

In [ ]:
def get_attention_map(model, input_ids, token_type_ids, attention_mask, output_attentions=False):
    _, _, attentions = model.bert(input_ids, token_type_ids, attention_mask, )
    return attentions


def _draw_logo_map(input_seq, attn, columns_aas, layer, head, output_dir, dir='row'):
    if dir=='row':
        dim=0
    elif dir=='column':
        dim=1
    else:
        raise Exception('Wrong direction')
    df_array = np.zeros((len(input_seq), 20))
    for j, aa in enumerate(input_seq):
        aa_ind = columns_aas.index(aa)
        df_array[j][aa_ind] = attn.mean(dim=dim)[j]
    df = pd.DataFrame(df_array)
    df.columns = columns_aas
    df.index = df.index + 1
    nn_logo = logomaker.Logo(df, color_scheme='chemistry', figsize=(20, 2.5), )
    plt.savefig(f'{output_dir}/logo_{layer}_{head}_{dir}.png', dpi=300)
    plt.close()


def _get_highlighted(df_array, input_seq, thrd=0.5):
    max_value = df_array.max()
    residues = []
    for ix in (df_array.sum(axis=1)>thrd*max_value).nonzero()[0]:
        residues.append(f'{ix+1}{input_seq[ix]}')
    return residues

def draw_logo_map(input_seq, attention, layer, head_num, output_dir):
    columns_aas = [
        'A', 'C', 'D', 'E',
        'F', 'G', 'H', 'I',
        'K', 'L', 'M', 'N',
        'P', 'Q', 'R', 'S',
        'T', 'V', 'W', 'Y'
    ]
    highlighted_residues = {}
    fig, axes = plt.subplots(8, sharex=True, figsize=(20, 2.5*head_num))

    for head in range(head_num):
        attn = attention[head][1:len(input_seq)+1, 1:len(input_seq)+1].detach()

        df_array = np.zeros((len(input_seq), 20))
        for j, aa in enumerate(input_seq):
            aa_ind = columns_aas.index(aa)
            df_array[j][aa_ind] = attn.mean(dim=0)[j]
#             df_array[j][aa_ind] = attn.max(dim=0)[0][j]
        df = pd.DataFrame(df_array)
        df.columns = columns_aas
        df.index = df.index + 1
        nn_logo = logomaker.Logo(df, color_scheme='chemistry', figsize=(20, 2.5), ax=axes[head])

        highlighted_residues[head] = _get_highlighted(df_array, input_seq, thrd=0.5)

    plt.savefig(f'{output_dir}/logo_{layer}.png', dpi=300)
    plt.show()
    plt.close()

    with open(f'{output_dir}/highlighted_residues_{layer}.txt', 'w') as fp:
        fp.write('Head\tResidues\n')
        for head, residues in highlighted_residues.items():
            residues.sort()
            residues = ';'.join(residues)
            fp.write(f'{head}\t{residues}\n')